In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display # Allows the use of display() for DataFrames
import seaborn as sb
%matplotlib inline
# from keras.applications.densenet import DenseNet121

import keras
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Input, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, Add, Concatenate
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
import timeit
import os
from datetime import datetime
import tensorflow as tf
from math import ceil
from training_mode import TrainingMode

from main_v2 import (generate_dataframe_from_csv_horizontal, generate_dataframe_from_csv_vertical, 
                         get_model_inputs, create_multi_generator, build_model, ConvBlock)
# https://github.com/keras-team/keras/issues/4161#issuecomment-366031228
# from keras.backend.tensorflow_backend import set_session
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
# config.log_device_placement = True  # to log device placement (on which device the operation ran)
#                                     # (nothing gets printed in Jupyter, only if you run it standalone)
# sess = tf.Session(config=config)
# set_session(sess)  # set this TensorFlow session as the default session for Keras

# import wandb
# from wandb.keras import WandbCallback
from train import TrainingRunner

Using TensorFlow backend.


In [2]:
# df = generate_dataframe_from_csv_vertical("train.csv", inputs=1)
# df = generate_dataframe_from_csv_horizontal("train.csv")
df = pd.read_csv("./train_controls_root.csv",  dtype={'sirna': object})
val_split = 0.25
epochs = 20
batch_size = 8
train_size = int(len(df)*(1-val_split))
valid_size = int(len(df)*(val_split))
steps = ceil(train_size/batch_size)
steps_valid = ceil(valid_size/batch_size)
print(train_size, valid_size, steps, steps_valid)
# y = df["sirna"]
# display(df)

6145 2048 769 256


In [2]:

#runner = TrainingRunner(filename="train_root.csv", use_multi=True, epochs=epochs, batch_size=batch_size, val_split=val_split, use_wandb=False)
# runner = TrainingRunner(batch_size=16, filename="train_root.csv", train_mode=TrainingMode.CELL_MULTI)
runner = TrainingRunner(batch_size=32, train_mode=TrainingMode.SINGLE, use_tb=False, epochs=20)
# runner = TrainingRunner(filename="train_controls_root.csv", batch_size=16, controls_only=True, train_mode=TrainingMode.MULTI)
# runner = TrainingRunner(filename="train_controls_root.csv", batch_size=16, controls_only=True, train_mode=TrainingMode.MULTI, weights_path="saved_models/weights.20190821-201844=180=2.hdf5")
# runner = TrainingRunner(filename="train_controls_root.csv", batch_size=16, controls_only=True, train_mode=TrainingMode.MULTI, weights_path="saved_models/weights.20190822-194205=16=2.43.hdf5")
# runner = TrainingRunner(filename="train_controls_root.csv", batch_size=16, controls_only=True, train_mode=TrainingMode.MULTI)
runner.summary()


Found 58424 validated image filenames belonging to 1108 classes.
Found 14606 validated image filenames belonging to 1108 classes.
Instructions for updating:
Colocations handled automatically by placer.


In [3]:
runner.run()
# bi = runner.train_generator.__getitem__(1)
# runner.evaluate() 
# [3.5677363468132928, 0.035387431360585725] ['loss', 'acc'] with no checkpoint loaded
# [2.4260314856454177, 0.369737644932705] ['loss', 'acc'] with checkpoint loaded 
# [2.46415597567695, 0.5991458206768804] ['loss', 'acc'] with better model

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 350, 350, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 350, 350, 32)      2432      
_________________________________________________________________
batch_normalization_1 (Batch (None, 350, 350, 32)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 175, 175, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 175, 175, 64)      18496     
_________________________________________________________________
batch_normalization_2 (Batch (None, 175, 175, 64)      256       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 87, 87, 64)        0         
__________

InternalError: GPU sync failed

In [5]:
MEANS = np.array([6.74696984, 14.74640167, 10.51260864, 10.45369445,  5.49959796, 9.81545561])
STDS = np.array([7.95876312, 12.17305868, 5.86172946, 7.83451711, 4.701167, 5.43130431])

In [6]:
np.shape(bi)

(16, 6, 224, 224, 3)

In [8]:
img1 = np.array(bi[0][0])
print(img1.shape)
imgdone = (img1 - MEANS[0]) / STDS[0]

(224, 224, 3)


In [10]:
# np.subtract(np.array(bi, np.float32), MEANS)
bidone = np.transpose(((np.array(np.transpose(np.array(bi, np.float32), axes=(0,2,3,4,1))) - MEANS) / STDS), axes=(4,0,1,2,3))
bidone.shape
# np.transpose(((np.transpose(np.array(bi, np.float32), axes=(0,2,3,4,1)) - MEANS) / STDS), axes=(0,4,1,2,3))
# np.array(np.transpose(np.array(bi, np.float32), axes=(1,0,2,3,4)))
# np.transpose(bi).shape
# np.subtract(bi.transpose(), MEANS.transpose())

(6, 16, 224, 224, 3)

In [11]:
bidone[0][0] == imgdone

array([[[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        ...,
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]],

       [[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        ...,
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]],

       [[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        ...,
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]],

       ...,

       [[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        ...,
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]],

       [[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        ...,
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,

In [ ]:
# runner.train_generator.__getitem__(1)
runner.run()

checkpoint path:  saved_models/weights.20190827-192518
Instructions for updating:
Use tf.cast instead.
Epoch 1/300
 43/410 [==>...........................] - ETA: 3:49 - loss: 4.2635 - acc: 0.0363

In [1]:
runner.model.save("saved_models\controls.model2.h5")
runner.model.save_weights("saved_models\controls.weights2.h5")
# runner.model.save("saved_models\cell_model.h5")

NameError: name 'runner' is not defined

In [4]:
runner.model.get_layer("dense_2").get_weights()

[array([[-6.1085071e-02, -1.0798127e-01,  1.5040853e-02, ...,
          2.0778170e-01,  7.2927117e-02, -4.5259718e-02],
        [-4.9801216e-02, -7.5902522e-02,  1.1211267e-01, ...,
         -1.4181557e-01,  2.9580915e-02,  7.7237003e-02],
        [-9.2577264e-02,  3.2547787e-01, -1.9574043e-01, ...,
          1.5512139e-01,  1.2781534e-01, -2.0233193e-01],
        ...,
        [-2.3858441e-02, -1.3601704e-04, -9.5326498e-02, ...,
         -1.3117903e-01, -4.6537083e-02,  1.7190069e-01],
        [ 2.3135018e-02,  2.9167417e-01, -2.4364465e-01, ...,
          3.7858848e-02, -7.9273723e-02, -5.2115779e-02],
        [ 4.5518849e-02, -2.4418640e-01,  2.0865026e-01, ...,
         -2.5177875e-01, -2.0243596e-01,  1.4429529e-01]], dtype=float32),
 array([ 0.11616212, -0.13406324,  0.13664906,  0.01222345,  0.02543289,
         0.06810106,  0.05293903,  0.14323905,  0.12143134, -0.20676266,
         0.02409862,  0.19136432, -0.43610457,  0.1265633 , -0.191939  ,
         0.04214715, -0.0781578

In [3]:
# wandb.init(project="testing-ml")

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, 
                                   verbose=1, mode='auto', min_delta=0.0001)

early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=15)

csv_logger = CSVLogger(filename='./training_log.csv',
                       separator=',',
                       append=True)

logdir = os.path.join("logs", datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)


(images, trainY) = get_model_inputs(df)

train_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=val_split)

# train_generator = create_multi_generator(df, train_datagen, "training") 
# valid_generator = create_multi_generator(df, train_datagen, "validation") 
train_generator = train_datagen.flow_from_dataframe(
        df,
        directory="./",
        x_col="img_path",
        y_col="sirna",
        target_size=(224, 224),
        batch_size=batch_size,
        subset="training",
        class_mode='categorical')

valid_generator = train_datagen.flow_from_dataframe(
        df,
        directory="./",
        x_col="img_path",
        y_col="sirna",
        target_size=(224, 224),
        batch_size=batch_size,
        subset="validation",
        class_mode='categorical')

Found 54773 validated image filenames belonging to 1108 classes.
Found 18257 validated image filenames belonging to 1108 classes.


In [4]:
model = build_model()
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      4864      
_________________________________________________________________
batch_normalization_1 (Batch (None, 224, 224, 64)      256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
batch_normalization_2

In [ ]:
#https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)
# WORKERS = 2

model.fit_generator(train_generator, 
                    steps_per_epoch=steps,
                    validation_data=valid_generator,
                    validation_steps=steps_valid,
#                     workers=WORKERS, 
#                     use_multiprocessing=True,
                    epochs=epochs, callbacks=[checkpointer,reduceLROnPlat, early, csv_logger, tensorboard_callback], verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
1712/1712 [==============================] - 678s 396ms/step - loss: 7.5768 - acc: 0.0012 - val_loss: 16.4520 - val_acc: 8.2160e-04

Epoch 00001: val_loss improved from inf to 16.45205, saving model to saved_models/weights.best.from_scratch.hdf5
Epoch 2/20
  41/1712 [..............................] - ETA: 9:26 - loss: 7.3360 - acc: 0.0023

In [ ]:
#!wandb run python train.py

In [ ]:
#tensorboard --logdir=logs/